<a href="https://colab.research.google.com/github/PravinTiwari023/Pytorch-Tutorial/blob/main/1_pytorch_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def dy_dx(x):
  return 2*x

x = 3
dy_dx(x)

6

In [2]:
import math

def dz_dx(x):
    return 2 * x * math.cos(x**2)

dz_dx(4)

-7.661275842587077

### **Torch Autograd**

In [3]:
import torch

x = torch.tensor(3.0, requires_grad=True)

y = x**2

print(x)
print(y)

tensor(3., requires_grad=True)
tensor(9., grad_fn=<PowBackward0>)


How does autograd calculate the gradient?
---
We have:

𝑦 = 𝑥^2

To find the gradient (derivative) of y with respect to x, we differentiate:

𝑑/𝑦 = 2𝑥

Since 𝑥 = 3, the gradient is:

2 × 3 = 6

In [4]:
y.backward()

print(x.grad)

tensor(6.)


**Key Takeaways**

* **requires_grad=True** enables automatic differentiation.
* **y.backward()** computes the derivative (gradient) of y with respect to x.
* **x.grad** stores the computed gradient.

In [5]:
n = float(input())

x = torch.tensor(n, requires_grad=True)

y = x**2

z = torch.sin(y)

4


In [6]:
print(x)
print(y)
print(z)

tensor(4., requires_grad=True)
tensor(16., grad_fn=<PowBackward0>)
tensor(-0.2879, grad_fn=<SinBackward0>)


In [7]:
z.backward()

In [8]:
print(x.grad)

tensor(-7.6613)


In [9]:
# For intermediate node grads wont be calculated
print(y.grad)

None


<ipython-input-9-5d87b478604e>:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(y.grad)


**Simple Nural Network Without Autograd**

In [10]:
import torch

# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [11]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [12]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [13]:
loss

tensor(6.7012)

In [14]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [15]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


**With Autograd**

In [16]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [17]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [18]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [19]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [20]:
loss.backward()

print(f"Autograd Gradient of loss w.r.t weight (dw): {w.grad}")
print(f"Autograd Gradient of loss w.r.t bias (db): {b.grad}")

Autograd Gradient of loss w.r.t weight (dw): 6.6917619705200195
Autograd Gradient of loss w.r.t bias (db): 0.9987704753875732


**Vector Data**

In [21]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [22]:
y.backward()

x.grad

tensor([0.6667, 1.3333, 2.0000])

In [23]:
# Clearing gradiant
x.grad.zero_()

tensor([0., 0., 0.])

**Dissabling Gradiant Tracking**

In [24]:
# option 1 - requires_grad_(False)
# option 2 - detach()
# option 3 - torch.no_grad()

**option 1**

In [25]:
x.requires_grad_(False)

tensor([1., 2., 3.])

In [26]:
x

tensor([1., 2., 3.])

In [29]:
y = x ** 2
y

tensor([1., 4., 9.])

**option 2**

In [30]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [31]:
z = x.detach()
z

tensor(2.)

In [32]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [33]:
y1 = z ** 2
y1

tensor(4.)

**option 3**

In [34]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [35]:
with torch.no_grad():
    y = x ** 2

y

tensor(4.)